In [5]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [7]:
path_train = "data/binarized_mnist_train.amat"
path_valid = "data/binarized_mnist_valid.amat"
path_test = "data/binarized_mnist_test.amat"

def data_loader(path):
    x = []
    with open(path_train) as file:
        #mat = list(map(int,img.readline().split()))
        for img in file.readlines():
            x.append(np.array(list(map(int,img.split()))).reshape(28,28))
    
    return torch.from_numpy(np.asarray(x)).float()

train = torch.utils.data.TensorDataset(data_loader(path_train))
trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)

test = torch.utils.data.TensorDataset(data_loader(path_train))
testloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=False)

valid = torch.utils.data.TensorDataset(data_loader(path_train))
validloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=False)

In [ ]:
class VAE(nn.Module, dim = 100, batch_size = 128):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.dim = dim
        self.batch_size = batch_size
        
        #ENCODER LAYERS
        encoder = nn.sequential(
            nn.conv2d(in_channels=1, out_channels=32, kernel_size=3),
            nn.ELU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2),
            nn.conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ELU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2),
            nn.conv2d(in_channels=64, out_channels=256, kernel_size=5),
            nn.ELU(),
            nn.Linear(256, 200)
        )
        
        #DECODER LAYERS
        decoder = nn.sequential(
            nn.Linear(100, 256),
            nn.ELU(),
            nn.conv2d(in_channels=256, out_channels=64, kernel_size=5, padding = 4),
            nn.ELU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.conv2d(in_channels=64, out_channels=32, kernel_size=3, padding = 2),
            nn.ELU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.conv2d(in_channels=32, out_channels=16, kernel_size=3, padding = 2),
            nn.ELU(),
            nn.conv2d(in_channels=16, out_channels=1, kernel_size=3, padding = 2)
        )

    def encode(self, x):
        q_params = encoder(x)
        mu = q_params[:, :self.dim]
        std = torch.exp(0.5*q_params[:, self.dim:])
        return mu, std

    def decode(self, mu, std):
        eps = torch.randn(self.batch_size, self.dim)
        return decoder(mu + eps*std)


    def forward(self, x):
        mu, std = self.encode(x)
        x_hat = decode(mu, std)
        return x_hat, mu, std
    
    def loss(x_hat, x, mu, std):
        BCE = F.binary_cross_entropy(x_hat, x, reduction='sum')
        KL = 0.5 * torch.sum(-1 - 2*log(std) + mu.pow(2) + std.pow(2))
        return BCE + KLD

    def train(self, trainloader, validloader, epoch):



model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
